In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf

# The array x_t will contain all result of X(t) in range  0<= t <=1500 
x_t = np.zeros(1506)
x_t[0]=1.5


# The function to calculate the result of x(t)
def Mackey_Glass(t_1):
  t = t_1 - 1 
  if t-25<0:
     xt_25 = 0 
  else: 
     xt_25 = x_t[t-25]  


  return ((x_t[t]) + ((0.2 * xt_25)/(1+ xt_25**10)) - (0.1 *x_t[t]))


# storing the result of X(t) in x_t
for i in range(1,1506):
  x_t[i] = Mackey_Glass(i)



inputs = np.zeros((1200, 5))   #shape is [1200,5]
outputs = np.zeros((1200, 1))  #shape is [1200,1]


#Generating the inputs and the outputs according to the lab instructions
#t satrts at 301 until it is 1500
t = 301
for i in range(1200):
  inputs[i][0] =  x_t[t-20]
  inputs[i][1] =  x_t[t-15]
  inputs[i][2] =  x_t[t-10]
  inputs[i][3] =  x_t[t-5]
  inputs[i][4] =  x_t[t]
  outputs[i][0] = x_t[t+5]
  t +=1

In [ ]:
t = np.arange(start=301, stop=1506, dtype=int)
plt.plot(t, x_t[301:1506])
plt.xlabel("t-values")
plt.ylabel("x(t) values")
plt.title("Time-series")
plt.show()

In [ ]:
# We divide the data into three subsets, trainning, validaion, and test  
x = np.split(inputs, [800,1000])
y = np.split(outputs, [800,1000])

x_train = np.array(x[0])           #800 (shape [800,5])
x_validation = np.array(x[1])      #200
x_test = np.array(x[2])            #200

y_train = np.array(y[0]).flatten()
y_validation = np.array(y[1]).flatten()
y_test = np.array(y[2]).flatten()

In [ ]:
#Building the network using tensorflow

class neural_network: 
  
   #n_layer is array that contain informatin about layers; number of nodes and the activation function in each layer
   def __init__(self, learning_rate, n_layer, epochs, loss, inputs, outputs):
    self.learning_rate = learning_rate
    self.n_layer = n_layer
    self.model=tf.keras.Sequential() 
    self.epochs = epochs
    self.loss = loss
    self.inputs= inputs
    self.outputs = outputs



   def createlayers(self):
     for layer in self.n_layer: 
       self.model.add(tf.keras.layers.Dense(layer[0] , activation=layer[1]))

   def train(self):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss') #Implenetaion of ealy stopping
    SGD = tf.keras.optimizers.SGD(learning_rate=self.learning_rate)       #
    self.model.compile(loss=self.loss, optimizer =  SGD)
    self.model.fit(self.inputs,
                   self.outputs, 
                   epochs = self.epochs,
                   validation_data=(x_validation, y_validation), 
                   callbacks=[early_stopping],
                   shuffle=False)
    

   def predict(self, x_test):
    return self.model.predict(x_test) 






my_neural_network =  neural_network(n_layer=[[5, "sigmoid"], [2, "sigmoid"], [1, "linear"]],
                                    epochs=100,
                                    loss="mse",
                                    inputs=x_train,
                                    outputs=y_train,
                                    learning_rate=0.4
                                  )



my_neural_network.createlayers()
my_neural_network.train()
temp = my_neural_network.predict(x_test)